# Exercises 1 - Representing Adversarial Game States

Note that this GameState can do any size and # of players (as long as they are positive).

In [407]:
import random

class GameState: 
    def __init__(self, num_players = 2, size = 3, print_each_move = True): 
        # We represent the Tic-Tac-Toe board as a 3x3 array. 
        # We use 0 to represent an empty square, 1 for Player 1's piece, and 2 for Player 2's piece. 
        self.board = []
        for row in range(size):
            self.board.append([])
            for col in range(size):
                self.board[row].append(0)
        # Player 1 goes first. 
        self.player_to_move = random.choice(list(range(1, num_players+1)))
        self.num_players = num_players
        self.size = size
        self.print_each_move = print_each_move
        # print(self)
        self.winner = 0
        self.done = False
 
    def make_move(self, row, col): 
        # Make a move at the specified location. 
        # This assumes that the move is valid. 
        assert self.board[row][col] == 0 
        self.board[row][col] = self.player_to_move 
        # Switch the player to move. 
        self.player_to_move += 1
        if self.player_to_move > self.num_players:
            self.player_to_move = 1
        done, winner = self.get_status()
        self.winner = winner
        self.done = done
 
    def get_valid_moves(self): 
        # Return a list of valid moves. 
        # A move is a tuple (row, col). 
        return [(row, col) for row in range(self.size) for col in range(self.size) if self.board[row][col] == 0] 
 
    def is_game_over(self): 
        # Check if the game is over. 
        # This could be done more efficiently, but for simplicity we just check all possibilities. 
        return self.done

    def get_status(self):
        # Quick check
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] == 0:
                    return False, 0 # Game not over yet

        for player in range(1, self.num_players + 1): 
            for row in range(self.size): 
                if all(self.board[row][col] == player for col in range(self.size)): 
                    return True, player 
            for col in range(self.size): 
                if all(self.board[row][col] == player for row in range(self.size)): 
                    return True, player 
            if all(self.board[i][i] == player for i in range(self.size)): 
                return True, player 
            if all(self.board[i][self.size-1-i] == player for i in range(self.size)): 
                return True, player 
        return True, 0 # Cat's game
    
    def print(self):
        print(f'{self.player_to_move}/{self.num_players}')
        for row in range(self.size):
            print(self.board[row])
    
    def get_winner(self):
        game_over, winner = is_game_over(self)
        return winner
    
    def play_game(self):
        print('Starting Game!')
        while True:
            if self.done:
                if self.winner == 0:
                    print("Cat's Game!")
                else:
                    print('Game over. Winner: ', self.winner)
                return
            
            vm = self.get_valid_moves()
            mv = random.choice(vm)
            print(f'Player {self.player_to_move} chose move {mv} from valid moves {vm}: ')
            self.make_move(*mv)
            if self.print_each_move:
                self.print()
            

gs = GameState(print_each_move=False)
gs.play_game()

        

Starting Game!
Player 1 chose move (0, 0) from valid moves [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]: 
Player 2 chose move (0, 1) from valid moves [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]: 
Player 1 chose move (2, 2) from valid moves [(0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]: 
Player 2 chose move (1, 0) from valid moves [(0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1)]: 
Player 1 chose move (2, 1) from valid moves [(0, 2), (1, 1), (1, 2), (2, 0), (2, 1)]: 
Player 2 chose move (1, 2) from valid moves [(0, 2), (1, 1), (1, 2), (2, 0)]: 
Player 1 chose move (0, 2) from valid moves [(0, 2), (1, 1), (2, 0)]: 
Player 2 chose move (2, 0) from valid moves [(1, 1), (2, 0)]: 
Player 1 chose move (1, 1) from valid moves [(1, 1)]: 
Game over. Winner:  1


# Exercises 2 - Minimax and Adversarial Game States

## Ex 1 - Copy() (and minimax(), evaluate(), __str__()

In [380]:

def evaluate(self): 
    # If the game is over, return the score from the point of view of the current player. 
    if self.is_game_over(): 
        if self.get_winner() == self.player_to_move: 
            return -1  # We lost 
        elif self.get_winner() is None: 
            return 0  # It's a draw 
        else: 
            return 1  # We won 
    else: 
        return 0  # If the game isn't over, the score is 0. 

def minimax(self, depth): 
    # If we've reached the maximum depth or the game is over, return the score. 
    if depth == 0 or self.is_game_over(): 
        return self.evaluate(), None 

    # Initialize the best score and best move. 
    if self.player_to_move == 1:  # Player 1 wants to maximize the score. 
        best_score = -float('inf') 
        sign = 1 
    else:  # Player 2 wants to minimize the score. 
        best_score = float('inf') 
        sign = -1 
    best_move = None 

    # Try all possible moves. 
    for move in self.get_valid_moves(): 
        new_state = self.copy()  # copy() is a helper method that clones the current state. 
        new_state.make_move(*move) 
        score, _ = new_state.minimax(depth - 1) 

        # Update the best score and best move. 
        if sign * score > sign * best_score: 
            best_score = score 
            best_move = move 

    return best_score, best_move 

# Test copying
a = [[1,2],[3,4]]
print(a)
b = a
print(b)
b[0][0]=5
print(a, b)
import copy as cp
c = cp.copy(b)
c[0][1] = 7
print(a,b,c)
d = cp.deepcopy(c)
d[1][0] = 9
print(a,b,c,d)


g1 = GameState()
print(str(g1), repr(g1))

print(str(a))


# Update print() to use a toString method
def game_state_str(self):
    s = f'{self.player_to_move}/{self.num_players}\n'
    for row in range(self.size):
       s += str(self.board[row])
       s += "\n" 
    return s

# augment or update methods of the GameState class
GameState.evaluate = evaluate
GameState.minimax = minimax
GameState.copy = cp.deepcopy
GameState.__str__ = game_state_str
GameState.print = lambda self: print(str(self))

# Verify that after copy, the boards are independent
g2 = GameState(print_each_move=False)
print(str(g2))
g3 = g2.copy()
g2.play_game()
print(g2, g3)
g3.play_game()
print(g2,g3)



[[1, 2], [3, 4]]
[[1, 2], [3, 4]]
[[5, 2], [3, 4]] [[5, 2], [3, 4]]
[[5, 7], [3, 4]] [[5, 7], [3, 4]] [[5, 7], [3, 4]]
[[5, 7], [3, 4]] [[5, 7], [3, 4]] [[5, 7], [3, 4]] [[5, 7], [9, 4]]
1/2
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[[5, 7], [3, 4]]
2/2
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

Starting Game!
Player 2 chose move (1, 2) from valid moves [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]: 
Player 1 chose move (0, 0) from valid moves [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1), (2, 2)]: 
Player 2 chose move (1, 0) from valid moves [(0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1), (2, 2)]: 
Player 1 chose move (0, 2) from valid moves [(0, 1), (0, 2), (1, 1), (2, 0), (2, 1), (2, 2)]: 
Player 2 chose move (2, 1) from valid moves [(0, 1), (1, 1), (2, 0), (2, 1), (2, 2)]: 
Player 1 chose move (0, 1) from valid moves [(0, 1), (1, 1), (2, 0), (2, 2)]: 
Player 2 chose move (2, 0) from valid moves [(1, 1), (2, 0), (2, 2)]: 
Player 1 chose move (1, 1) from valid moves [(1

## Ex 1b - Adversarial without pruning

## Ex 2 - Alpha/Beta Pruning

## Ex 3 - Adversarial *WITH* Pruning

# Exercises 3 - Minimax with Alpha-Beta Pruning

# Exercises 4 - Monte Carlo Tree Search in Adversarial Game States